In [ ]:
# import os
# os.environ["ORCHESTRA_DEPLOYMENT_TYPE"] = "remote"
# os.environ["DEV_MODE"] = "True"
# os.environ["TEST_EXTERNAL_REGISTRY"] = "k3d-registry.localhost:5800"
# !pip install aiosmtpd
# !uv pip install aiosmtpd

In [ ]:
# stdlib
import os
import re

environment = os.environ.get("ORCHESTRA_DEPLOYMENT_TYPE", "python")
environment

In [ ]:
# third party
from helpers import SENDER
from helpers import create_smtp_test_server
from helpers import create_user
from helpers import emails_table
from helpers import make_user

# syft absolute
import syft as sy

In [ ]:
smtp_server = create_smtp_test_server()

In [ ]:
server = sy.orchestra.launch(
    name="bigquery-high",
    dev_mode=True,
    server_side_type="high",
    reset=True,
    port="8080",
    n_consumers=1,  # How many workers to be spawned
    create_producer=True,  # Can produce more workers
)

In [ ]:
high_client = sy.login(
    url="http://localhost:8080", email="info@openmined.org", password="changethis"
)

In [ ]:
high_client.api.services.settings.enable_notifications(
    email_sender=SENDER,
    email_server="localhost",
    email_port="1025",
)
high_client.api.services.settings.disable_notifications()  # Turn off OnBoard Email Notifications

In [ ]:
num_users = 5
users = []
email_disable_index = 0
for i in range(num_users):
    user = make_user()
    create_user(high_client, user)
    user.client = high_client
    if email_disable_index == i:
        user.email_disabled = True
    users.append(user)

In [ ]:
pattern = r"syft_client\.reset_password\(token='(.*?)', new_password=.*?\)"

for user in users:
    user.client = high_client
    if user.email_disabled:
        high_client.api.services.settings.disable_notifications()
    else:
        high_client.api.services.settings.enable_notifications()

    user.client.forgot_password(email=user.email)

    if user.email_disabled:
        assert emails_table.get(user.email, None) is None
    else:
        assert emails_table.get(user.email, None) is not None
        recv_email = emails_table.get(user.email, None)
        assert recv_email.email_from == SENDER
        assert recv_email.email_to[0] == user.email

        token = re.search(pattern, recv_email.email_content).group(1)
        new_password = user.make_new_password()
        result = user.client.reset_password(token=token, new_password=new_password)
        assert isinstance(result, sy.SyftSuccess)

In [ ]:
server.land()
smtp_server.stop()